In [ ]:
"""SMOTE is not implemented properly, train on oversampled data, validate on original data ratio"""

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import ensemble
from scipy.stats import spearmanr

import time
import warnings
warnings.filterwarnings("ignore")

import lightgbm as lgb
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedKFold

import utils

In [3]:
df_train = utils.read_pickles('../data/input/smote')

100%|██████████| 3/3 [00:01<00:00,  1.94it/s]


In [4]:
target = df_train.pop('target')

In [5]:
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,5.7470,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
1,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,8.0851,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
2,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,5.9525,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
3,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,8.2450,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
4,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,7.6784,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [8]:
test_file = '../data/input/test.csv'

df_test = pd.read_csv(test_file)
test_id = df_test.pop('ID_code')

In [7]:

params = {'num_leaves': 9,
         'min_data_in_leaf': 42,
         'objective': 'binary',
         'max_depth': 16,
         'learning_rate': 0.0123,
         'boosting': 'gbdt',
         'bagging_freq': 5,
         'bagging_fraction': 0.8,
         'feature_fraction': 0.8201,
         'bagging_seed': 11,
         'reg_alpha': 1.728910519108444,
         'reg_lambda': 4.9847051755586085,
         'random_state': 42,
         'metric': 'auc',
         'verbosity': -1,
         'subsample': 0.81,
         'min_gain_to_split': 0.01077313523861969,
         'min_child_weight': 19.428902804238373,
         'num_threads': 4}

In [9]:
%%time
fold_n=3

folds = StratifiedKFold(n_splits=fold_n, shuffle=True, random_state=30)
y_pred_lgb = np.zeros(len(df_test))

for fold_n, (train_index, valid_index) in enumerate(folds.split(df_train, target)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = df_train.iloc[train_index], df_train.iloc[valid_index]
    y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
        
    lgb_model = lgb.train(params, train_data, num_boost_round=20000,
                    valid_sets = [train_data, valid_data],verbose_eval=300,early_stopping_rounds = 200)
            
    y_pred_lgb += lgb_model.predict(df_test, num_iteration=lgb_model.best_iteration)/3

Fold 0 started at Thu Feb 28 19:37:49 2019
Training until validation scores don't improve for 200 rounds.
[300]	training's auc: 0.902204	valid_1's auc: 0.895634
[600]	training's auc: 0.926824	valid_1's auc: 0.919637
[900]	training's auc: 0.937308	valid_1's auc: 0.929657
[1200]	training's auc: 0.944171	valid_1's auc: 0.936147
[1500]	training's auc: 0.948883	valid_1's auc: 0.940565
[1800]	training's auc: 0.952679	valid_1's auc: 0.944181
[2100]	training's auc: 0.955695	valid_1's auc: 0.947088
[2400]	training's auc: 0.958185	valid_1's auc: 0.949441
[2700]	training's auc: 0.960327	valid_1's auc: 0.951438
[3000]	training's auc: 0.962104	valid_1's auc: 0.953104
[3300]	training's auc: 0.963631	valid_1's auc: 0.954467
[3600]	training's auc: 0.964989	valid_1's auc: 0.955712
[3900]	training's auc: 0.966183	valid_1's auc: 0.95674
[4200]	training's auc: 0.967251	valid_1's auc: 0.957687
[4500]	training's auc: 0.96817	valid_1's auc: 0.958444
[4800]	training's auc: 0.969019	valid_1's auc: 0.95919
[510

In [10]:
submission_lgb_smote_2 = pd.DataFrame({
        "ID_code": test_id,
        "target": y_pred_lgb
    })
submission_lgb_smote_2.to_csv('../results/submission_lgb_smote.csv', index=False)